In [1]:
import pandas as pd
import numpy as np
import optuna

In [2]:
xgb_df = pd.read_csv("/kaggle/input/xgbtuned/submission_xgb.csv")
lgb_df = pd.read_csv("/kaggle/input/lightboost/lgsubmission (1).csv")
cat_df = pd.read_csv("/kaggle/input/catboostt/submission_catb.csv")
lr_df  = pd.read_csv("/kaggle/input/linearregression/lrsubmission.csv")
gb_df  = pd.read_csv("/kaggle/input/adaboost/adabosubmission.csv")

xgb_df = xgb_df.rename(columns={'Premium Amount': 'xgb_pred'})
lgb_df = lgb_df.rename(columns={'Premium Amount': 'lgb_pred'})
cat_df = cat_df.rename(columns={'Premium Amount': 'cat_pred'})
lr_df  = lr_df.rename(columns={'Premium Amount': 'lr_pred'})
gb_df  = gb_df.rename(columns={'Premium Amount': 'gb_pred'})


In [3]:
gb_df.head()

,id,gb_pred
0,1200000,627.084775
1,1200001,737.329060
2,1200002,742.021768
3,1200003,745.040577
4,1200004,723.644087


In [4]:
lr_df.head()

,id,lr_pred
0,1200000,907.222200
1,1200001,582.835247
2,1200002,674.658230
3,1200003,643.867759
4,1200004,696.832252


In [5]:
lgb_df.head()

,id,lgb_pred
0,1200000,718.528380
1,1200001,863.284798
2,1200002,799.298493
3,1200003,789.131510
4,1200004,778.381459


In [6]:
xgb_df.head()

,id,xgb_pred
0,1200000,719.940637
1,1200001,820.888477
2,1200002,791.860571
3,1200003,792.193005
4,1200004,772.765942


In [7]:
cat_df.head()

,id,cat_pred
0,1200000,778.918317
1,1200001,784.389178
2,1200002,785.537065
3,1200003,782.219267
4,1200004,764.650581


In [8]:
combined_df = pd.concat([xgb_df, lgb_df, cat_df, lr_df, gb_df], axis = 1)
combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]
combined_df.head()

,id,xgb_pred,lgb_pred,cat_pred,lr_pred,gb_pred
0,1200000,719.940637,718.528380,778.918317,907.222200,627.084775
1,1200001,820.888477,863.284798,784.389178,582.835247,737.329060
2,1200002,791.860571,799.298493,785.537065,674.658230,742.021768
3,1200003,792.193005,789.131510,782.219267,643.867759,745.040577
4,1200004,772.765942,778.381459,764.650581,696.832252,723.644087


In [9]:
test_df = combined_df.copy()
test_df.head()

,id,xgb_pred,lgb_pred,cat_pred,lr_pred,gb_pred
0,1200000,719.940637,718.528380,778.918317,907.222200,627.084775
1,1200001,820.888477,863.284798,784.389178,582.835247,737.329060
2,1200002,791.860571,799.298493,785.537065,674.658230,742.021768
3,1200003,792.193005,789.131510,782.219267,643.867759,745.040577
4,1200004,772.765942,778.381459,764.650581,696.832252,723.644087


In [10]:
import numpy as np
import pandas as pd

# Assuming your DataFrame is called 'df'
def exponential_mean_log1p(predictions):
    # Apply log1p transformation
    log_preds = np.log1p(predictions)
    
    # Calculate mean of log-transformed predictions
    mean_log_preds = np.mean(log_preds, axis=1)
    
    # Apply exponential (inverse of log1p)
    final_predictions = np.expm1(mean_log_preds)
    
    return final_predictions


# Select prediction columns
pred_columns = ['xgb_pred', 'lgb_pred', 'cat_pred', 'lr_pred', 'gb_pred']

# Calculate final predictions
combined_df['final_pred'] = exponential_mean_log1p(combined_df[pred_columns])

# Display the results
combined_df.head()

,id,xgb_pred,lgb_pred,cat_pred,lr_pred,gb_pred,final_pred
0,1200000,719.940637,718.528380,778.918317,907.222200,627.084775,744.832642
1,1200001,820.888477,863.284798,784.389178,582.835247,737.329060,751.001991
2,1200002,791.860571,799.298493,785.537065,674.658230,742.021768,757.192221
3,1200003,792.193005,789.131510,782.219267,643.867759,745.040577,748.271040
4,1200004,772.765942,778.381459,764.650581,696.832252,723.644087,746.572328


In [11]:
import numpy as np
import pandas as pd

def average_predictions(df, pred_columns):
    return np.mean(df[pred_columns], axis=1)

pred_columns = ['xgb_pred', 'lgb_pred', 'cat_pred', 'lr_pred', 'gb_pred']

# Calculate average predictions
combined_df['avg_predictions'] = average_predictions(combined_df, pred_columns)

# Display results
combined_df.head()

,id,xgb_pred,lgb_pred,cat_pred,lr_pred,gb_pred,final_pred,avg_predictions
0,1200000,719.940637,718.528380,778.918317,907.222200,627.084775,744.832642,750.338862
1,1200001,820.888477,863.284798,784.389178,582.835247,737.329060,751.001991,757.745352
2,1200002,791.860571,799.298493,785.537065,674.658230,742.021768,757.192221,758.675225
3,1200003,792.193005,789.131510,782.219267,643.867759,745.040577,748.271040,750.490424
4,1200004,772.765942,778.381459,764.650581,696.832252,723.644087,746.572328,747.254864


In [12]:
sub = pd.read_csv('/kaggle/input/playground-series-s4e12/sample_submission.csv')
sub['Premium Amount'] = combined_df['final_pred']
sub.to_csv('submission.csv', index=False)
sub.head()

,id,Premium Amount
0,1200000,744.832642
1,1200001,751.001991
2,1200002,757.192221
3,1200003,748.271040
4,1200004,746.572328


In [13]:
xgb_oof_df = pd.read_csv("/kaggle/input/xgbtuned/oof_xgb.csv").drop(columns =['Actual','Fold'])
lgb_oof_df = pd.read_csv("/kaggle/input/lightboost/oof_lgbm.csv").drop(columns =['Actual','Fold'])
cat_oof_df = pd.read_csv("/kaggle/input/catboostt/oof_catb.csv").drop(columns =['Fold'])
lr_oof_df = pd.read_csv("/kaggle/input/linearregression/oof_lr.csv").drop(columns =['Actual','Fold'])
gb_oof_df = pd.read_csv("/kaggle/input/adaboost/oof_adabo.csv").drop(columns =['Actual','Fold'])

In [14]:
gb_oof_df.head()

,ID,OOF_Pred
0,4,738.200705
1,15,720.541774
2,21,718.366317
3,30,747.705190
4,34,738.200705


In [15]:
xgb_oof_df.head()

,ID,OOF_Pred_XGB
0,1,718.714111
1,14,779.824036
2,23,709.281189
3,27,802.486633
4,30,822.166992


In [16]:
print(gb_oof_df.columns)


Index(['ID', 'OOF_Pred'], dtype='object')


In [17]:
# Merge DataFrames on 'ID'
combined_oof_df = xgb_oof_df.merge(lgb_oof_df, on='ID')
combined_oof_df = combined_oof_df.merge(cat_oof_df, on='ID')
combined_oof_df = combined_oof_df.merge(lr_oof_df, on='ID')  # Also merge the LR predictions
combined_oof_df = combined_oof_df.merge(gb_oof_df, on="ID")




# Select the required columns
combined_oof_df = combined_oof_df[['ID', 'OOF_Pred_XGB', 'OOF_Pred_LGB', 'OOF_Pred_cat', 'OOF_Pred_LR', 'OOF_Pred', 'Actual']]

# Display the final combined DataFrame
combined_oof_df.head()


,ID,OOF_Pred_XGB,OOF_Pred_LGB,OOF_Pred_cat,OOF_Pred_LR,OOF_Pred,Actual
0,1,718.714111,763.030032,668.446737,660.500120,651.797271,1483.0
1,14,779.824036,795.237141,835.617504,823.017541,741.631335,30.0
2,23,709.281189,630.072633,729.990639,601.261028,697.527291,3869.0
3,27,802.486633,815.339673,772.072172,727.567159,740.549301,2680.0
4,30,822.166992,835.328327,803.697458,789.055631,747.705190,641.0


In [18]:
train_df = combined_oof_df.copy()
train_df.head()

,ID,OOF_Pred_XGB,OOF_Pred_LGB,OOF_Pred_cat,OOF_Pred_LR,OOF_Pred,Actual
0,1,718.714111,763.030032,668.446737,660.500120,651.797271,1483.0
1,14,779.824036,795.237141,835.617504,823.017541,741.631335,30.0
2,23,709.281189,630.072633,729.990639,601.261028,697.527291,3869.0
3,27,802.486633,815.339673,772.072172,727.567159,740.549301,2680.0
4,30,822.166992,835.328327,803.697458,789.055631,747.705190,641.0


In [19]:
combined_oof_df.head()

,ID,OOF_Pred_XGB,OOF_Pred_LGB,OOF_Pred_cat,OOF_Pred_LR,OOF_Pred,Actual
0,1,718.714111,763.030032,668.446737,660.500120,651.797271,1483.0
1,14,779.824036,795.237141,835.617504,823.017541,741.631335,30.0
2,23,709.281189,630.072633,729.990639,601.261028,697.527291,3869.0
3,27,802.486633,815.339673,772.072172,727.567159,740.549301,2680.0
4,30,822.166992,835.328327,803.697458,789.055631,747.705190,641.0


In [20]:
# import numpy as np
# import pandas as pd
# import optuna
# from sklearn.model_selection import train_test_split

# def rmsle(y_true, y_pred):
#     """
#     Root Mean Squared Logarithmic Error
#     Handles cases with zero and negative values
#     """
#     # Add small constant to avoid log(0)
#     y_true = np.abs(y_true) + 1e-6
#     y_pred = np.abs(y_pred) + 1e-6
    
#     # Calculate log error
#     log_error = np.log1p(y_true) - np.log1p(y_pred)
    
#     # Calculate RMSLE
#     return np.sqrt(np.mean(log_error**2))

# def objective(trial):
#     # Prepare the data
#     X = df[['OOF_Pred_XGB', 'OOF_Pred_LGB', 'OOF_Pred_cat', 'OOF_Pred_LR', 'OOF_Pred']]  # Include all model predictions
#     y = df['Actual']
    
#     # Split the data
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
#     # Optimize weights
#     w1 = trial.suggest_float('w1', 0, 1)
#     w2 = trial.suggest_float('w2', 0, 1)
#     w3 = trial.suggest_float('w3', 0, 1)
#     w4 = trial.suggest_float('w4', 0, 1)
#     w5 = trial.suggest_float('w5', 0, 1)  # Added weight for GB predictions
    
#     # Normalize weights
#     total_weight = w1 + w2 + w3 + w4 + w5  # Total weight now includes w5 for GB
#     weights = [w1/total_weight, w2/total_weight, w3/total_weight, w4/total_weight, w5/total_weight]  # Include GB weight
    
#     # Calculate weighted prediction
#     pred = np.average(X_test, axis=1, weights=weights)
    
#     # Calculate RMSLE
#     rmsle_score = rmsle(y_test, pred)
    
#     return rmsle_score

# # Merge DataFrames on 'ID'
# combined_oof_df = xgb_oof_df.merge(lgb_oof_df, on='ID')
# combined_oof_df = combined_oof_df.merge(cat_oof_df, on='ID')
# combined_oof_df = combined_oof_df.merge(lr_oof_df, on='ID')  # Also merge the LR predictions
# combined_oof_df = combined_oof_df.merge(gb_oof_df, on='ID')  # Merge GB predictions

# # Select the required columns
# combined_oof_df = combined_oof_df[['ID', 'OOF_Pred_XGB', 'OOF_Pred_LGB', 'OOF_Pred_cat', 'OOF_Pred_LR', 'OOF_Pred', 'Actual']]

# # Display the final combined DataFrame
# print(combined_oof_df.head())

# # Create DataFrame (as in your previous example)
# df = combined_oof_df

# # Create a study object and optimize
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=500)

# # Print results
# print("Best trial:")
# trial = study.best_trial
# print("  Best RMSLE: ", trial.value)
# print("  Best Weights:")
# print(f"    XGB: {trial.params['w1']}")
# print(f"    LGB: {trial.params['w2']}")
# print(f"    Cat: {trial.params['w3']}")
# print(f"    LR: {trial.params['w4']}")
# print(f"    GB: {trial.params['w5']}")  # Print GB weight

# # Final weighted prediction using best weights
# best_w1 = trial.params['w1']
# best_w2 = trial.params['w2']
# best_w3 = trial.params['w3']
# best_w4 = trial.params['w4']
# best_w5 = trial.params['w5']  # Get best weight for GB
# total_weight = best_w1 + best_w2 + best_w3 + best_w4 + best_w5  # Total weight includes GB
# weights = [best_w1/total_weight, best_w2/total_weight, best_w3/total_weight, best_w4/total_weight, best_w5/total_weight]

# # Final prediction using best weights
# final_pred = np.average(df[['OOF_Pred_XGB', 'OOF_Pred_LGB', 'OOF_Pred_cat', 'OOF_Pred_LR', 'OOF_Pred']], 
#                         axis=1, 
#                         weights=weights)
# df['Weighted_Prediction'] = final_pred

# # Display final results
# print("\nFinal DataFrame with Weighted Prediction:")
# print(df)


In [21]:
# weights

NameError: name 'weights' is not defined

In [ ]:
# XGB =  weights[0]
# LGB =  weights[1]
# Cat = weights[2]
# LR  = weights[3]
# GB  = weights[4]


In [ ]:
# # Compute the weighted predictions
# weighted_predicted = (
#     (combined_df["xgb_pred"] * weights[0]) +
#     (combined_df["lgb_pred"] * weights[1]) +
#     (combined_df["cat_pred"] * weights[2]) +
#     (combined_df["lr_pred"] * weights[3]) +
#     (combined_df["gb_pred"] * weights[4])
# )

In [ ]:
# combined_df['weighted_prediction'] = weighted_predicted
# combined_df.head()

In [ ]:
# sub = pd.read_csv('/kaggle/input/playground-series-s4e12/sample_submission.csv')
# sub['Premium Amount'] = combined_df['weighted_prediction']
# sub.to_csv('submission(05).csv', index=False)
# sub.head()

In [22]:
train_df.head()

,ID,OOF_Pred_XGB,OOF_Pred_LGB,OOF_Pred_cat,OOF_Pred_LR,OOF_Pred,Actual
0,1,718.714111,763.030032,668.446737,660.500120,651.797271,1483.0
1,14,779.824036,795.237141,835.617504,823.017541,741.631335,30.0
2,23,709.281189,630.072633,729.990639,601.261028,697.527291,3869.0
3,27,802.486633,815.339673,772.072172,727.567159,740.549301,2680.0
4,30,822.166992,835.328327,803.697458,789.055631,747.705190,641.0


In [23]:
test_df.head()

,id,xgb_pred,lgb_pred,cat_pred,lr_pred,gb_pred
0,1200000,719.940637,718.528380,778.918317,907.222200,627.084775
1,1200001,820.888477,863.284798,784.389178,582.835247,737.329060
2,1200002,791.860571,799.298493,785.537065,674.658230,742.021768
3,1200003,792.193005,789.131510,782.219267,643.867759,745.040577
4,1200004,772.765942,778.381459,764.650581,696.832252,723.644087


In [24]:
 # Renaming all columns
renamed_columns = {
    "id": "ID",
    "xgb_pred": "XGB",
    "lgb_pred": "LGB",
    "cat_pred": "CAT",
    "lr_pred": "LR",
    "gb_pred": "ADA",
}

# Rename columns
test_df = test_df.rename(columns=renamed_columns)

# Display the result
test_df.head()

,ID,XGB,LGB,CAT,LR,ADA
0,1200000,719.940637,718.528380,778.918317,907.222200,627.084775
1,1200001,820.888477,863.284798,784.389178,582.835247,737.329060
2,1200002,791.860571,799.298493,785.537065,674.658230,742.021768
3,1200003,792.193005,789.131510,782.219267,643.867759,745.040577
4,1200004,772.765942,778.381459,764.650581,696.832252,723.644087


In [25]:
 # Renaming all columns
renamed_columns = {
    "ID": "ID",
    "OOF_Pred_XGB": "XGB",
    "OOF_Pred_LGB": "LGB",
    "OOF_Pred_cat": "CAT",
    "OOF_Pred_LR": "LR",
    "OOF_Pred": "ADA",
    "Actual": "Actual",
}

# Rename columns
train_df = train_df.rename(columns=renamed_columns)

# Display the result
train_df.head()

,ID,XGB,LGB,CAT,LR,ADA,Actual
0,1,718.714111,763.030032,668.446737,660.500120,651.797271,1483.0
1,14,779.824036,795.237141,835.617504,823.017541,741.631335,30.0
2,23,709.281189,630.072633,729.990639,601.261028,697.527291,3869.0
3,27,802.486633,815.339673,772.072172,727.567159,740.549301,2680.0
4,30,822.166992,835.328327,803.697458,789.055631,747.705190,641.0


In [26]:
train_df.head()

,ID,XGB,LGB,CAT,LR,ADA,Actual
0,1,718.714111,763.030032,668.446737,660.500120,651.797271,1483.0
1,14,779.824036,795.237141,835.617504,823.017541,741.631335,30.0
2,23,709.281189,630.072633,729.990639,601.261028,697.527291,3869.0
3,27,802.486633,815.339673,772.072172,727.567159,740.549301,2680.0
4,30,822.166992,835.328327,803.697458,789.055631,747.705190,641.0


In [27]:
test_df.head()

,ID,XGB,LGB,CAT,LR,ADA
0,1200000,719.940637,718.528380,778.918317,907.222200,627.084775
1,1200001,820.888477,863.284798,784.389178,582.835247,737.329060
2,1200002,791.860571,799.298493,785.537065,674.658230,742.021768
3,1200003,792.193005,789.131510,782.219267,643.867759,745.040577
4,1200004,772.765942,778.381459,764.650581,696.832252,723.644087


In [28]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_log_error
import xgboost as xgb
import optuna

# Log transform function
def log_transform(y):
    return np.log1p(y)  # log(1+y) to handle zero values

# Inverse log transform function
def inverse_log_transform(y_log):
    return np.expm1(y_log)  # exp(y) - 1 to revert log1p

# Prepare data
X = train_df.drop('Actual', axis=1)
y = train_df['Actual']

# Log transform the target variable
y_log = log_transform(y)

def objective(trial):
    # Hyperparameter space to search
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 10000, step=100),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 1e-8, 10.0, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        'eval_metric': 'rmsle',
        "objective": "reg:squarederror",
        "tree_method": "hist",
        "device": "cuda",
        "early_stopping_rounds": 50,
        "enable_categorical": True
    }
    
    # Split data into train and validation sets
    X_train, X_valid, y_train, y_valid = train_test_split(X, y_log, test_size=0.2, random_state=42)
    
    # Initialize the model with the given parameters
    model = xgb.XGBRegressor(**params)
    
    # Fit the model on log-transformed target
    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        verbose=False
    )
    
    # Predict on the validation set and inverse transform
    y_pred_log = model.predict(X_valid)
    y_pred = inverse_log_transform(y_pred_log)
    y_valid_original = inverse_log_transform(y_valid)
    
    # Calculate RMSLE on original scale
    score = np.sqrt(mean_squared_log_error(y_valid_original, y_pred))
    
    return score

# Optuna Study for Hyperparameter Tuning
study = optuna.create_study(direction="minimize")  # We want to minimize RMSLE
study.optimize(objective, n_trials=50)

# Print the best parameters and score
print("Best RMSLE:", study.best_value)
print("Best Hyperparameters:", study.best_params)

[I 2024-12-31 03:00:32,274] A new study created in memory with name: no-name-6a19ca0a-de06-405e-8756-903635f6091f
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [03:00:35] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
[I 2024-12-31 03:00:35,303] Trial 0 finished with value: 1.0433357202613065 and parameters: {'n_estimators': 4600, 'learning_rate': 0.03358843992199035, 'max_depth': 4, 'min_child_weight': 9, 'gamma': 1.4040898894825681e-06, 'subsample': 0.6229890898726547, 'colsample_bytree': 0.9997369271839527, 'reg_alpha': 2.2921223876552997e

Best RMSLE: 1.0432551746733927
Best Hyperparameters: {'n_estimators': 3100, 'learning_rate': 0.05224601623306532, 'max_depth': 5, 'min_child_weight': 4, 'gamma': 7.781562767035464e-08, 'subsample': 0.8400007296826764, 'colsample_bytree': 0.7052355766819175, 'reg_alpha': 0.07193757571180548, 'reg_lambda': 0.4849578951814768}


In [29]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from xgboost import XGBRegressor
from numpy import log1p, expm1

# Set random seed
np.random.seed(42)

# Assume train_df and test_df are your DataFrames
df = train_df

# Features and target
X = df.drop('Actual', axis=1)
y = df['Actual']

# Log transform the target
y_log = np.log1p(y)

# Stratify target by binning into discrete intervals
binner = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
y_binned_log = binner.fit_transform(y_log.values.reshape(-1, 1)).astype(int).ravel()

# Stratified K-Fold
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Arrays to store predictions
oof_predictions = np.zeros(len(X))  # Out-of-fold predictions
test_predictions = np.zeros(len(X))  # Test predictions for each fold

# Test set
X_test = test_df
test_preds_per_fold = np.zeros((len(X_test), n_splits))  # Store test predictions per fold
fold_oof_results = []  # To store ID, target, and OOF predictions

# XGBoost Regressor parameters
model_params = {
    'n_estimators': 3100,
    'learning_rate': 0.05224601623306532,
    'max_depth': 5,
    'min_child_weight': 4,
    'gamma': 7.781562767035464e-08,
    'subsample': 0.8400007296826764,
    'colsample_bytree': 0.7052355766819175,
    'reg_alpha': 0.07193757571180548,
    'reg_lambda': 0.4849578951814768,
    "random_state": 42,
    'eval_metric': 'rmse',
    "objective": "reg:squarederror",  # XGBoost objective for regression
    "tree_method": "hist",  # Use GPU acceleration
    "device": "cuda",
    "enable_categorical": True  # Enable categorical handling if required
}

# Cross-validation loop
for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y_binned_log)):
    print(f"Fold {fold + 1}/{n_splits}")
    
    # Split data
    X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    y_train, y_valid = y_log.iloc[train_idx], y_log.iloc[valid_idx]
    
    # Model
    model = XGBRegressor(**model_params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        early_stopping_rounds=300,
        verbose=0
    )
    
    # Predictions (log transformed)
    log_oof_preds = model.predict(X_valid)
    
    # Revert log transformation
    oof_predictions[valid_idx] = np.expm1(log_oof_preds)

    # Calculate RMSLE for the fold
    fold_rmsle = np.sqrt(mean_squared_log_error(y.iloc[valid_idx], oof_predictions[valid_idx]))
    print(f"Fold {fold + 1} RMSLE: {fold_rmsle:.4f}")
    
    # Store fold results
    fold_result = pd.DataFrame({
        'ID': X.index[valid_idx],
        'Actual': y.iloc[valid_idx],
        'OOF_Pred_XGB': oof_predictions[valid_idx],
        'Fold': fold + 1
    })
    fold_oof_results.append(fold_result)
    
    # Test set predictions
    log_test_preds = model.predict(X_test)
    test_preds_per_fold[:, fold] = np.expm1(log_test_preds)

# Combine fold results
oof_results_df = pd.concat(fold_oof_results, axis=0, ignore_index=True)

# Average predictions on test data
final_test_predictions = test_preds_per_fold.mean(axis=1)

# Evaluate OOF predictions
oof_mse = mean_squared_error(y, oof_predictions)
oof_rmsle = np.sqrt(mean_squared_log_error(y, oof_predictions))

print(f"OOF Mean Squared Error: {oof_mse:.4f}")
print(f"OOF Root Mean Squared Log Error: {oof_rmsle:.4f}")

# Output predictions
print("Final Test Predictions:", final_test_predictions)
print(oof_results_df.head())


Fold 1/5


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Fold 1 RMSLE: 1.0455
Fold 2/5


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Fold 2 RMSLE: 1.0440
Fold 3/5


...

Fold 3 RMSLE: 1.0471
Fold 4/5


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Fold 4 RMSLE: 1.0453
Fold 5/5


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Fold 5 RMSLE: 1.0439
OOF Mean Squared Error: 850843.9066
OOF Root Mean Squared Log Error: 1.0452
Final Test Predictions: [773.28479004 833.61499023 804.32042236 ... 837.42200928 800.29519043
 799.5871582 ]
   ID  Actual  OOF_Pred_XGB  Fold
0   1    30.0    784.164368     1
1   2  3869.0    686.713196     1
2   3  2680.0    812.932617     1
3   4   641.0    828.611328     1
4   6  1128.0    828.890259     1


In [33]:
sub = pd.read_csv('/kaggle/input/playground-series-s4e12/sample_submission.csv')
sub['Premium Amount'] = final_test_predictions
sub.to_csv('submission_meta.csv', index=False)
oof_results_df.to_csv('oof_xgb.csv',index = False)
sub.head()


,id,Premium Amount
0,1200000,773.284790
1,1200001,833.614990
2,1200002,804.320422
3,1200003,799.820117
4,1200004,776.885461
